# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2015-06-20,Automatic image classification and retrieval b...,Journal of Zhejiang A&F University,We proposed a novel conceptual framework for d...,"<b>Yao Fei</b>, Ye Kang, Zhou Jianhua<sup>*</s...",Automatic-image-classification-and-retrieval-b...,http://zlxb.zafu.edu.cn/CN/abstract/abstract10...
1,2016-10-01,VIIRS-based remote sensing estimation of groun...,Remote Sensing of Environment,We proposed a novel spatiotemporal statistical...,"Wu Jiansheng, <b>Yao Fei</b>, Li Weifeng<sup>*...",VIIRS-based-remote-sensing-estimation-of-groun...,https://www.sciencedirect.com/science/article/...
2,2017-07-22,Saturation Correction for Nighttime Lights Dat...,Remote Sensing,We proposed a new method to correct the satura...,"Wang Zheng, <b>Yao Fei</b>, Li Weifeng, Wu Jia...",Saturation-Correction-for-Nighttime-Lights-Dat...,https://www.mdpi.com/2072-4292/9/7/759
3,2018-03-15,A multidimensional comparison between MODIS an...,Science of The Total Environment,This is one of the earliest studies that revea...,"<b>Yao Fei</b>, Si Menglin, Li Weifeng<sup>*</...",A-multidimensional-comparison-between-MODIS-an...,https://www.sciencedirect.com/science/article/...
4,2019-04-08,Estimating Daily PM<sub>2.5</sub> Concentratio...,Remote Sensing,We further developed a nested spatiotemporal s...,"<b>Yao Fei</b>, Wu Jiansheng<sup>*</sup>, Li W...",Estimating-Daily-PM25-Concentrations-in-Beijin...,https://www.mdpi.com/2072-4292/11/7/841
5,2019-05-01,A spatially structured adaptive two-stage mode...,ISPRS Journal of Photogrammetry and Remote Sen...,We proposed a spatially structured adaptive tw...,"<b>Yao Fei</b>, Wu Jiansheng<sup>*</sup>, Li W...",A-spatially-structured-adaptive-two-stage-mode...,https://www.sciencedirect.com/science/article/...
6,2019-09-24,Impacts of AOD Correction and Spatial Scale on...,Remote Sensing,"By employing GF-1-retrieved AOD, we extended t...","Wu Jiansheng, Liang Jingtian, Zhou Liguo, <b>Y...",Impacts-of-AOD-Correction-and-Spatial-Scale-on...,https://www.mdpi.com/2072-4292/11/19/2223


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
#     md += "\nRecommended citation: " + item.citation # This is unnecessary and hence commented. (fei 03/02/2020)
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [6]:
!ls ../_publications/

2015-06-20-Automatic-image-classification-and-retrieval-by-analyzing-plant-leaf-features.md
2016-10-01-VIIRS-based-remote-sensing-estimation-of-ground-level-PM25-concentrations-in-Beijing-Tianjin-Hebei-A-spatiotemporal-statistical-model.md
2017-07-22-Saturation-Correction-for-Nighttime-Lights-Data-Based-on-the-Relative-NDVI.md
2018-03-15-A-multidimensional-comparison-between-MODIS-and-VIIRS-AOD-in-estimating-ground-level-PM25-concentrations-over-a-heavily-polluted-region-in-China.md
2019-04-08-Estimating-Daily-PM25-Concentrations-in-Beijing-Using-750-m-VIIRS-IP-AOD-Retrievals-and-a-Nested-Spatiotemporal-Statistical-Model.md
2019-05-01-A-spatially-structured-adaptive-two-stage-model-for-retrieving-ground-level-PM25-concentrations-from-VIIRS-AOD-in-China.md
2019-09-24-Impacts-of-AOD-Correction-and-Spatial-Scale-on-the-Correlation-between-High-Resolution-AOD-from-Gaofen-1-Satellite-and-In-Situ-PM25-Measurements-in-Shenzhen-City-China.md


In [7]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

cat: ../_publications/2009-10-01-paper-title-number-1.md: No such file or directory
